# Overview

This is the **fifth notebook** in my machine learning series.  
In the [previous notebook](https://www.kaggle.com/code/rameelsohail/k-nearest-neighbors), we implemented the **K-Nearest Neighbors (KNN)** algorithm.

In this notebook, we'll be implementing our first **clustering** algorithm — **K-Means**.  
We’ll be using the classic **Iris dataset** to explore how K-Means groups similar data points together without labeled outputs.

Let's get started!

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go

# Data Loading and Analysis

## Dataset — Iris 

In this notebook, we'll be using the **Iris dataset**, one of the most famous datasets in machine learning.  

The **Iris dataset** was first introduced in **R.A. Fisher’s 1936 paper**, *“The Use of Multiple Measurements in Taxonomic Problems”*, and is also available on the **UCI Machine Learning Repository**.  

It contains **three species of Iris flowers**, with **50 samples each**, and includes measurements of their physical attributes.  
One of the species is **linearly separable**, while the other two are **not linearly separable**, making this dataset a great starting point for classification tasks.  

### Columns:
- **Id** — Unique identifier for each observation  
- **SepalLengthCm** — Length of the sepal (in cm)  
- **SepalWidthCm** — Width of the sepal (in cm)  
- **PetalLengthCm** — Length of the petal (in cm)  
- **PetalWidthCm** — Width of the petal (in cm)  
- **Species** — Type of Iris flower (Setosa, Versicolor, Virginica)


In [2]:
# Load the dataset
df = pd.read_csv('Iris.csv')
df.drop('Id', axis=1, inplace=True) # Drop redundant columns

In [3]:
# Setting the features
X = df.iloc[:,:-1]

# Setting the labels
y = df.iloc[:,-1]

In [4]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Data Visualization

In [5]:
fig = px.pie(df, 'Species', template='plotly_dark', title='Data Distribution',color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

This plot shows that there is an **equal number of samples for each class**, making the dataset **balanced**.

### Sepal Length 

In [6]:
fig = px.box(df, x='Species', y='SepalLengthCm', color='Species', template='plotly_dark',color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [7]:
fig = px.histogram(df, x='SepalLengthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'], nbins=50)
fig.show(renderer='iframe')

These plots show that:  
- **Setosa** flowers are much smaller, making them **linearly separable** from the other two classes.  
- **Virginica** flowers are generally the largest and contain a outlier.  
- It is difficult to distinguish between **Versicolor** and **Virginica**, as their features overlap.
- The amount of overlap between **Versicolor** and **Virginica** make this a less useful feature. 

### Petal Length

In [8]:
fig = px.box(df, x='Species', y='PetalLengthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [9]:
fig = px.histogram(df, x='PetalLengthCm', color='Species', template='plotly_dark', nbins=30, color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

These plots show that:
- Again **Setosa** has a much smaller **Petal Length** comapared to the other two classes.
- **Virginica** has the largest **Petal Length.
- There is some overlap between **Versicolor** and **Virginica**, regardless of that **Petal Length** shows some differentiation between the classes.

### Sepal Width

In [10]:
fig = px.box(df, x='Species', y='SepalWidthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [11]:
fig = px.histogram(df, x='SepalWidthCm', color='Species', template='plotly_dark', nbins=30, color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

These plots show that:
- **Setosa** has the largest **Sepal width**.
- **Versicolor** has the smallest **Sepal width**.
- There is alot of overlap between the classes, showing that **Sepal Width** might not be a useful feature.

### Petal Width

In [12]:
fig = px.box(df, x='Species', y='PetalWidthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [13]:
fig = px.histogram(df, x='PetalWidthCm', color='Species', template='plotly_dark', nbins=20, color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

These plots show that:
- **Setosa** has much smaller **PetalWidth** than the other 2 classes, it also has 2 outliers.
- Again the difference is less clear between **Virginica** and **Versicolor**
- Overall this seems like an PetalWidth might be useful in making good predictions.

In [14]:
fig = px.scatter(df, x='SepalLengthCm', y='SepalWidthCm', size='PetalLengthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [15]:
fig = px.scatter(df, x='SepalLengthCm', y='SepalWidthCm', size='PetalWidthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [16]:
fig = px.scatter(df, x='PetalLengthCm', y='PetalWidthCm', size='SepalLengthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

In [17]:
fig = px.scatter(df, x='PetalLengthCm', y='PetalWidthCm', size='SepalWidthCm', color='Species', template='plotly_dark', color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'])
fig.show(renderer='iframe')

These plots show us that:
- **Setosa** is the smaller of the three flower species.
- **Virginica** is the largest of the three flower species.
- **Versicolor** and **Virginica** show overlap , making them difficult to distinguish.
- There is a positive correlation between some of the features.

# Model Implementation

## What is K-Means?

**K-Means Clustering** is an **unsupervised machine learning** algorithm that groups data points into clusters based on their similarity.  
Unlike **supervised learning**, where models are trained on labeled data, **K-Means** is used when the dataset is **unlabeled** (though the Iris dataset does contain labels, we’ll ignore them for learning purposes).  
The goal is to uncover hidden patterns or natural groupings within the data.

**Example:**  
An online store can use K-Means to segment customers into groups such as **“Budget Shoppers,” “Frequent Buyers,”** and **“Big Spenders”** based on their purchasing behavior.

---

## How does K-Means work?

Suppose we have a dataset consisting of items, each represented by a vector of feature values.  
Our goal is to categorize these items into groups based on similarity. The parameter **k** represents the number of clusters we want to form.

K-Means groups the data into **k clusters** by minimizing the **distance** between each data point and its assigned cluster centroid.  
The **Euclidean distance** is commonly used as the similarity measure.

The algorithm follows these main steps:

1. **Initialization:** Randomly select *k* initial cluster centroids.  
2. **Assignment Step:** Assign each data point to the nearest centroid, forming clusters.  
3. **Update Step:** Recalculate each centroid as the mean of all points assigned to that cluster.  
4. **Repeat:** Continue steps 2–3 until the centroids stop changing or a maximum number of iterations is reached.

---

### Initialization
We start by randomly selecting *k* points from the dataset as the **initial centroids**.  
These act as the centers of the clusters in the first iteration.

---

### Assignment Step
For each data point in our training set, we determine which centroid it is closest to. This is done b0y measuring the distance between the data point and each centroid and selecting the centroid with the smallest distance as the closest one. We use an index notation, denoted as  $c^{(i)}$, to represent the index of the closest centroid to the data point $x^{(i)}$

---

### 🔁 Computing Cluster Means
Once all data points are assigned to their nearest centroids, we recompute the **centroid of each cluster** as the mean of all points belonging to that cluster.  
Mathematically, for each cluster \( j \):

$$
\mu_j = \frac{1}{|C_j|} \sum_{i \in C_j} x^{(i)}
$$

The **objective (cost) function** that K-Means minimizes is the **total within-cluster variance** (or **sum of squared distances**) between data points and their assigned centroids:

$$
\sum_{i=1}^{m} \| x^{(i)} - \mu_j \|^2
$$

**where**:
- $k$: number of clusters  
- $m$: number of data points  
- $x^{(i)}$: the *i-th* data point  
- $\mu_j$: the centroid of cluster *j*  

---

The algorithm repeats the assignment and recomputation of centroids until convergence, where the centroids no longer change significantly or a specified number of iterations is reached.

In [21]:
class Kmeans:
    """
    K means clustering implementation
    Parameters:
    K(int) : Number of independent clusters(centroids)

    Attributes:
    __init__(self, K, max_iters=10) : Initializes the number of centroids and maximum iterations.
    initialze_centroids(self, X) : Initializes the centroids to K random points in the data.
    assign_centroids(self, X) : Assigns each data point to the nearest centroid.
    compute_mean(self, X, points) : Computes the mean distance of all the points belonging to a specific centroid.
    fit(self, X) : Driver of our Kmeans class.
    """
    def __init__(self, K, max_iters=10):
        assert K > 0, "K must be greater than 0."
        assert max_iters > 0, "The number of max iterations must be greater than 0."
        self.K = K
        self.max_iters = max_iters
        self.centroids = None

    def initialize_centroids(self, X):
        assert X.shape[0] >= self.K, "There must be atleast K samples in our dataset."
        
        random_idx = np.random.permutation(X.shape[0])
        centroids_idx = random_idx[:self.K]
        self.centroids = X[centroids_idx]

    def assign_centroids(self, X):
        """
            Assign each point in our dataset to the closest centroid.

            Parameters:
                X (np.ndarray) : Samples

            Returns:
                points (np.ndarray) : Array consisting of indexes of the closest centroid to each sample in the dataset
        """
        points = []
        X = np.expand_dims(X, axis=1)
        distances = np.linalg.norm(X-self.centroids, axis=-1)
        points = np.argmin(distances, axis=1)

        assert points.shape[0] == X.shape[0], "The number points must be equal to the samples."
        return points

    def compute_mean(self, X, points):
        """
        Compute the mean of all the samples belonging to a particular centroid.

        Parameters:
        X (np.ndarray) : Samples
        points (np.ndarray) : Array consisting of indexes of the closest centroid to each sample in the dataset

        Returns:
        Centroids (np.ndarray) : Array consisting of the newly calculated centroids.
        """

        centroids = np.zeros([self.K, X.shape[1]])
        for i in range(self.K):
            centroids_mean = X[i==points].mean(axis=0)
            centroids[i] = centroids_mean

        return centroids

    def fit(self, X):
        """
        Cluster the dataset using the K-Means algorithm.
        
        Parameters:
        X (numpy.ndarray): dataset to cluster
        
        Returns:
        numpy.ndarray: array containing the final centroids for each cluster
        numpy.ndarray: array containing the index of the centroid for each point
        """

        self.initialize_centroids(X) # initialize the centroids
        for i in range(self.max_iters):
            points = self.assign_centroids(X) # assign each sample to the closest centroid
            self.centroids = self.compute_mean(X, points) # update the centroids based on the mean of the current points in the cluster

            assert len(self.centroids) == self.K, "Number of centroids should equal K."
            assert X.shape[1] == self.centroids.shape[1], "Dimensionality of centroids should match input data."
            assert max(points) < self.K, "Cluster index should be less than K."
            assert min(points) >= 0, "Cluster index should be non-negative."
            
        return self.centroids, points

# Evaluation

In [19]:
X = X.values

In [22]:
kmeans = Kmeans(3, 1000)
centroids, points = kmeans.fit(X)

In [43]:
fig = go.Figure()

# Cluster 1
fig.add_trace(go.Scatter(
    x=X[points==0, 0], y=X[points==0, 1],
    mode='markers',marker_color='#491D8B',name='Iris-setosa'
    ))

# Cluster 2
fig.add_trace(go.Scatter(
    x=X[points==1, 0], y=X[points==1, 1],
    mode='markers', marker_color='#7D3AC1',name='Iris-versicolor'
    ))

# Cluster 3
fig.add_trace(go.Scatter(
    x=X[points==2, 0], y=X[points==2, 1],
    mode='markers', marker_color='#EB548C',name='Iris-virginica'
    ))

# Centroids
fig.add_trace(go.Scatter(
    x=centroids[:, 0], y=centroids[:,1],
    mode='markers',marker_color='white',marker_symbol=4,marker_size=13,name='Centroids'
))
fig.update_layout(template='plotly_dark', height=500, width=1000)
fig.show(renderer='iframe')

# Thank You
If anyone has any suggestion, please do let me know.